## 이메일 내용으로부터 구조화된 정보 추출

<div style="text-align: right"> Initial issue : 2025.05.06 </div>
<div style="text-align: right"> last update : 2025.05.06 </div>

In [1]:
email_conversation = """From: 유광명 (kmyu@kmtune.com)
To: 건호 대리님 (gh@kmtune.me)
Subject: RAG 솔루션 시연 관련 미팅 제안

안녕하세요, 건호 대리님,

저는 kmtune의 유광명입니다. 최근 귀사에서 AI를 활용한 혁신적인 솔루션을 모색 중이라는 소식을 들었습니다. kmtune AI 및 RAG 솔루션 분야에서 다양한 경험과 노하우를 가진 기업으로, 귀사의 요구에 맞는 최적의 솔루션을 제공할 수 있다고 자부합니다.

저희 kmtune의 RAG 솔루션은 귀사의 데이터 활용을 극대화하고, 실시간으로 정확한 정보 제공을 통해 비즈니스 의사결정을 지원하는 데 탁월한 성능을 보입니다. 이 솔루션은 특히 다양한 산업에서의 성공적인 적용 사례를 통해 그 효과를 입증하였습니다.

귀사와의 협력 가능성을 논의하고, 저희 RAG 솔루션의 구체적인 기능과 적용 방안을 시연하기 위해 미팅을 제안드립니다. 다음 주 목요일(7월 18일) 오전 10시에 귀사 사무실에서 만나 뵐 수 있을까요?

미팅 시간을 조율하기 어려우시다면, 편하신 다른 일정을 알려주시면 감사하겠습니다. 건호 대리님과의 소중한 만남을 통해 상호 발전적인 논의가 이루어지길 기대합니다.

감사합니다.

유광명
kmtune AI 솔루션팀"""

In [2]:
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
load_dotenv()

True

In [3]:
# 이메일 본문으로부터 주요 엔티티 추출
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    company: str = Field(description="메일을 보낸 사람의 회사 정보")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [4]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
output_parser = PydanticOutputParser(pydantic_object=EmailSummary)
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

#QUESTION:
다음의 이메일 내용 중에서 주요 내용을 추출해 주세요.

#EMAIL CONVERSATION:
{email_conversation}

#FORMAT:
{format}
"""    
)
prompt = prompt.partial(format=output_parser.get_format_instructions())

chain 생성 및 실행

In [5]:
chain = prompt | llm | output_parser
answer = chain.invoke({"email_conversation": email_conversation})
print(answer)

person='유광명' company='kmtune' email='kmyu@kmtune.com' subject='RAG 솔루션 시연 관련 미팅 제안' summary='유광명은 kmtune의 AI 및 RAG 솔루션을 소개하며, 귀사의 데이터 활용을 극대화하고 비즈니스 의사결정을 지원할 수 있는 솔루션을 제안합니다. 다음 주 목요일(7월 18일) 오전 10시에 미팅을 제안하며, 다른 일정이 필요하면 알려달라고 요청합니다.' date='7월 18일 오전 10시'


In [6]:
print(answer.summary)

유광명은 kmtune의 AI 및 RAG 솔루션을 소개하며, 귀사의 데이터 활용을 극대화하고 비즈니스 의사결정을 지원할 수 있는 솔루션을 제안합니다. 다음 주 목요일(7월 18일) 오전 10시에 미팅을 제안하며, 다른 일정이 필요하면 알려달라고 요청합니다.


In [7]:
answer

EmailSummary(person='유광명', company='kmtune', email='kmyu@kmtune.com', subject='RAG 솔루션 시연 관련 미팅 제안', summary='유광명은 kmtune의 AI 및 RAG 솔루션을 소개하며, 귀사의 데이터 활용을 극대화하고 비즈니스 의사결정을 지원할 수 있는 솔루션을 제안합니다. 다음 주 목요일(7월 18일) 오전 10시에 미팅을 제안하며, 다른 일정이 필요하면 알려달라고 요청합니다.', date='7월 18일 오전 10시')

### SERP API를 활용한 정보 검색의 활용

참고: https://serpapi.com/integrations/python

![serp.png](../../images/serp.png)

In [8]:
from langchain_community.utilities import SerpAPIWrapper

params = {"engine": "google", "gl": "kr", "hl": "ko", "num": "3"}
search = SerpAPIWrapper(params=params)

In [9]:
search.run("한국 추천 여행지")

[{'title': '부산광역시',
  'link': 'https://www.google.com/search?num=3&sca_esv=008f9e8726f85704&hl=ko&gl=kr&q=%EB%B6%80%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C&si=APYL9bu1Sl4M4TWndGcDs6ZL5WJXWNYEL_kgEEwAe0iMZIocdcRcFfR5TqXjkZttINvaoRc1Uefnw_TR2Hz6MKlIpf8fXRmKKg%3D%3D&sa=X&ved=2ahUKEwiZoaD9z42NAxWPOTQIHYMeE1gQs4ILegQIHRAD',
  'description': '해운대 해수욕장, 절, 산',
  'hotel_price': '₩57,807',
  'extracted_hotel_price': 57807,
  'thumbnail': 'https://serpapi.com/searches/68195cc1822c5f509e2a1757/images/708b910fc819221b2631187b170c20f37ba15c98b47a514b14356c7843616971914ee91c7ff4602c26c3df0d5eb3170b.jpeg'},
 {'title': '경주시',
  'link': 'https://www.google.com/search?num=3&sca_esv=008f9e8726f85704&hl=ko&gl=kr&q=%EA%B2%BD%EC%A3%BC%EC%8B%9C&si=APYL9btMsmZl0P9CyeA1NmMZFYv4xkDb-_Q4WCJadY9pxozSRWhLknP4of3VddAA7-mCf1c1F7i7gx_KEpcgOwnbUTok5BDq6g%3D%3D&sa=X&ved=2ahUKEwiZoaD9z42NAxWPOTQIHYMeE1gQs4ILegQIJhAD',
  'description': '불국사 및 석굴암',
  'hotel_price': '₩81,398',
  'extracted_hotel_price': 81398,
  'thumbnail': 

In [10]:
search_result = search.run("부산 여행지 site:naver.com")
search_result

"['그래서 오늘은 그런 분들을 위해 대표적인 부산 여행지 및 부산 명소 몇몇 곳을 골라 정리해 볼까 합니다. \\u200b. 1. 감천문화마을. 부산 사하구 감내2 ...', '카테고리 이동 돈 없는 여행 중독자 (˘ᵕ˘) ˚₊‧ · 1. 흰여울문화마을 · 2. 블루라인파크 · 3. 더베이 101 · 4. 감천문화마을 · 5. 광안리 해수욕장 · 6.', '일정이 짧다면 해운대, 오륙도, 남포동과 같은 핵심 관광지 위주로 여행하는 것이 좋고 2박 이상이라면 부산 외곽에 있는 기장, 낙동강 부근도 여행코스에 ...']"

In [11]:
type(search_result)

str

- 결과가 문자열이다.  
- 타입을 리스트로 변경하려면 eval 함수를 사용하면 된다.

In [12]:
search_result = eval(search_result)
type(search_result)

list

결과를 하나로 합치려면 다음을 사용

In [13]:
# 검색 결과
search_result_string = "\n".join(search_result)
search_result_string


'그래서 오늘은 그런 분들을 위해 대표적인 부산 여행지 및 부산 명소 몇몇 곳을 골라 정리해 볼까 합니다. \u200b. 1. 감천문화마을. 부산 사하구 감내2 ...\n카테고리 이동 돈 없는 여행 중독자 (˘ᵕ˘) ˚₊‧ · 1. 흰여울문화마을 · 2. 블루라인파크 · 3. 더베이 101 · 4. 감천문화마을 · 5. 광안리 해수욕장 · 6.\n일정이 짧다면 해운대, 오륙도, 남포동과 같은 핵심 관광지 위주로 여행하는 것이 좋고 2박 이상이라면 부산 외곽에 있는 기장, 낙동강 부근도 여행코스에 ...'

In [ ]:
search_result = search.run("유광명명")
search_result = eval(search_result)
search_result_string = "\n".join(search_result)
search_result_string


'천룡회주 철위강의 제자이며 천룡상회를 이끌고 있다. 때문에 유광명의 주 활동 무대는 무림이 아닌 상계가 될 것이며, 상계에서는 팔황중 절반 이상을 ...\n455화에선 노재수를 먼저 보내고 이진성과 싸울 준비를 하면서 김기명의 MMA스승이라는 것이 밝혀졌다. 이진성에게 기습적인 보디블로를 시작으로 여러 타격을 시전하지만 ...\n752 유광명지의(有光明之意). 괘사. 天心月光 正照萬里 천심월광 정조만리. 恒時積德 事多成就 항시적덕 사다성취. 或有膝憂 祈禱可免 혹유슬우 기도 ...'

## 구조화된 답변을 다음 체인의 입력으로 추가하기

In [15]:
report_prompt = PromptTemplate.from_template(
    """당신은 이메일의 주요 정보를 바탕으로 요약 정리해 주는 전문가 입니다.
당신의 임무는 다음의 이메일 정보를 바탕으로 보고서 형식의 요약을 작성하는 것입니다.
주어진 정보를 기반으로 양식(format)에 맞추어 요약을 작성해 주세요.

#Information:
- Sender: {sender}
- Additional Information about sender: {additional_information}
- Company: {company}
- Email: {email}
- Subject: {subject}
- Summary: {summary}
- Date: {date}

#Format(in markdown format):
🙇‍♂️ 보낸 사람:
- (보낸 사람의 이름, 회사 정보)

📧 이메일 주소:
- (보낸 사람의 이메일 주소)

😍 보낸 사람과 관련하여 검색된 추가 정보:
- (검색된 추가 정보)

✅ 주요 내용:
- (이메일 제목, 요약)

⏰ 일정:
- (미팅 날짜 및 시간)

#Answer:"""
)

In [16]:
report_chain = (
    report_prompt | ChatOpenAI(model="gpt-4-turbo", temperature=0) | StrOutputParser()
)

In [17]:
report_response = report_chain.invoke(
    {
        "sender": answer.person,
        "additional_information": search_result_string,
        "company": answer.company,
        "email": answer.email,
        "subject": answer.subject,
        "summary": answer.summary,
        "date": answer.date,
    }
)

In [18]:
print(report_response)

🙇‍♂️ 보낸 사람:
- 유광명, kmtune

📧 이메일 주소:
- kmyu@kmtune.com

😍 보낸 사람과 관련하여 검색된 추가 정보:
- 유광명은 천룡회주 철위강의 제자이며 천룡상회를 이끌고 있다. 상계에서 팔황중 절반 이상의 영향력을 가지고 있으며, 455화에서는 노재수를 먼저 보내고 이진성과 싸울 준비를 하면서 김기명의 MMA 스승으로 밝혀졌다. 또한, 752화에서는 유광명지의(有光明之意) 괘사와 천심월광 정조만리, 항시적덕 사다성취 등의 내용이 나온다.

✅ 주요 내용:
- 제목: RAG 솔루션 시연 관련 미팅 제안
- 요약: 유광명은 kmtune의 AI 및 RAG 솔루션을 소개하며, 귀사의 데이터 활용을 극대화하고 비즈니스 의사결정을 지원할 수 있는 솔루션을 제안합니다. 다음 주 목요일(7월 18일) 오전 10시에 미팅을 제안하며, 다른 일정이 필요하면 알려달라고 요청합니다.

⏰ 일정:
- 7월 18일 오전 10시
